In [ ]:
#@title Install required libraries
!pip -q install transformers accelerate sentence-transformers --upgrade
!pip -q install python-docx pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 83.3 MB/s eta 0:00:00


In [ ]:
#@title Runtime config
EMBED_MODEL_NAME    = "sentence-transformers/paraphrase-MiniLM-L6-v2" #@param ["sentence-transformers/paraphrase-MiniLM-L6-v2", "intfloat/multilingual-e5-base", "BAAI/bge-m3"] {allow-input: true}
LLM_MODEL_NAME      = "K-intelligence/Midm-2.0-Base-Instruct"
MAX_NEW_TOKENS_LLM  = 900   # 필요에 따라 조정

print("LLM :", LLM_MODEL_NAME)
print("EMB :", EMBED_MODEL_NAME)


LLM : K-intelligence/Midm-2.0-Base-Instruct
EMB : sentence-transformers/paraphrase-MiniLM-L6-v2


In [ ]:
#@title Adapters: HF LLM & HF Embedding
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import torch
from typing import List


# --- Replace your HFLLMAdapter with this Midm-safe version ---
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, re

class HFLLMAdapter:
    """
    Works with K-intelligence/Midm-2.0-Base-Instruct (and most chatty Causal LMs).
    - Removes token_type_ids if present (Midm doesn't use it)
    - Uses chat template when available
    - Deterministic decoding (do_sample=False)
    """
    def __init__(self, model_name: str = "K-intelligence/Midm-2.0-Base-Instruct", max_new_tokens: int = 900):
        self.model_name = model_name
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto",
            trust_remote_code=True,
        )
        self.max_new_tokens = max_new_tokens

        # Some models have no pad_token; use eos as pad to avoid warnings.
        if self.model.config.pad_token_id is None and self.tok.eos_token_id is not None:
            self.model.config.pad_token_id = self.tok.eos_token_id

        # whether we can format with chat template
        self._has_chat_template = hasattr(self.tok, "apply_chat_template")

    def _format_input(self, prompt: str) -> str:
        if self._has_chat_template:
            # System + user messages → add_generation_prompt=True makes the model continue as assistant
            msgs = [
                {"role": "system", "content": "You are a JSON-only generator. Return ONLY a single JSON object."},
                {"role": "user", "content": prompt},
            ]
            return self.tok.apply_chat_template(
                msgs, tokenize=False, add_generation_prompt=True
            )
        # Fallback: plain prompt
        return prompt

    def complete(self, prompt: str, **gen_kwargs) -> str:
        input_text = self._format_input(prompt)
        inputs = self.tok(input_text, return_tensors="pt")
        # 🚫 Midm doesn't accept token_type_ids → drop it
        inputs.pop("token_type_ids", None)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        out = self.model.generate(
            **inputs,
            do_sample=False,
            max_new_tokens=gen_kwargs.get("max_tokens", self.max_new_tokens),
            eos_token_id=self.tok.eos_token_id,
            pad_token_id=self.model.config.pad_token_id,
        )
        text = self.tok.decode(out[0], skip_special_tokens=True)

        # If the decoded text still contains the prompt prefix, strip it.
        if text.startswith(input_text):
            text = text[len(input_text):].lstrip()

        # Best-effort: if it didn't start with JSON, try to extract the outer {...}
        m = re.search(r"\{.*\}", text, flags=re.DOTALL)
        return m.group(0) if m else text


class HFSentenceEmbedAdapter:
    """
    embedding 모듈의 EmbeddingClient(embed)와 호환.
    - 기본 모델은 소형/빠른 ST 모델.
    """
    def __init__(self, model_name="sentence-transformers/paraphrase-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed(self, texts: List[str], **kwargs) -> List[List[float]]:
        embs = self.model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
        return [e.tolist() for e in embs]


In [ ]:
#@title Upload documents to ingest (txt, docx, pdf, hwpx 등)
from google.colab import files
from pathlib import Path

print("인제스트할 문서 파일들을 선택하세요.")
uploaded_docs = files.upload()  # 여러 개 선택 가능
doc_paths = []
for name, data in uploaded_docs.items():
    p = Path(name)
    with open(p, "wb") as f:
        f.write(data)
    doc_paths.append(str(p.resolve()))

print("✅ 업로드 완료:", doc_paths)


인제스트할 문서 파일들을 선택하세요.


Saving sample.txt to sample.txt
✅ 업로드 완료: ['/content/sample.txt']


In [ ]:
#@title Run full pipeline on uploaded documents
import importlib, json, time
from pathlib import Path

file_reader   = importlib.import_module("file_reader")
text_chunker  = importlib.import_module("text_chunker")
embedding_mod = importlib.import_module("embedding")

# 모델 초기화
llm = HFLLMAdapter(LLM_MODEL_NAME, max_new_tokens=MAX_NEW_TOKENS_LLM)
embedder = HFSentenceEmbedAdapter(EMBED_MODEL_NAME)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:


all_records = []
t0 = time.time()

for path in doc_paths:
    print(f"\n=== Processing: {path} ===")
    # 1) 파일 로드
    try:
        text = file_reader.read_file(path)
    except Exception as e:
        print(f"[ERR] read_file failed: {e}")
        continue

    if not isinstance(text, str) or not text.strip():
        print("[WARN] Empty text; skipping.")
        continue
    print(f"[OK] Loaded text: {len(text)} chars")

    # 2) LLM 기반 청크 분할
    try:
        chunk_result = text_chunker.llm_guided_sentence_chunk(
            text,
            llm=llm,
            max_sentences_per_chunk=20,
            system_hint="논리/주제 전환 기준으로 5~20문장씩 묶고, 오직 JSON만 출력하세요."
        )
    except Exception as e:
        print(f"[ERR] chunking failed: {e}")
        continue

    if not hasattr(chunk_result, "chunks") or len(chunk_result.chunks) == 0:
        print("[WARN] No chunks; skipping.")
        continue

    print(f"[OK] Chunks: {len(chunk_result.chunks)}")

    # 3) 임베딩
    chunks = [c.text for c in chunk_result.chunks]
    offsets = [{
        "chunk_index": c.chunk_index,
        "char_start": c.char_start,
        "char_end": c.char_end,
        "word_start": c.word_start,
        "word_end": c.word_end
    } for c in chunk_result.chunks]

    try:
        records = embedding_mod.build_embeddings(
            chunks=chunks,
            file_path=path,
            filetype=Path(path).suffix.lstrip(".").lower() or "unknown",
            offsets=offsets,
            embedder=embedder
        )
    except Exception as e:
        print(f"[ERR] embedding failed: {e}")
        continue

    print(f"[OK] Embeddings: {len(records)} vectors, dim={len(records[0].vector) if records else 0}")
    all_records.extend(records)

t1 = time.time()
print(f"\n=== DONE: {len(all_records)} records from {len(doc_paths)} files in {t1-t0:.2f}s ===")

# 4) 저장 (JSONL)
out_path = Path("embeddings.jsonl")
with open(out_path, "w", encoding="utf-8") as f:
    for r in all_records:
        f.write(json.dumps({
            "vector": r.vector,
            "text": r.text,
            "meta": {
                "file_path": r.meta.file_path,
                "filetype": r.meta.filetype,
                "chunk_index": r.meta.chunk_index,
                "char_start": r.meta.char_start,
                "char_end": r.meta.char_end,
                "word_start": r.meta.word_start,
                "word_end": r.meta.word_end
            }
        }, ensure_ascii=False) + "\n")

print(f"📝 Saved embeddings to: {out_path.resolve()}")


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== Processing: /content/sample.txt ===
[OK] Loaded text: 2167 chars
=== RAW LLM OUTPUT (first try) ===
{"chunks":[{"first_sentence_index":0,"last_sentence_index":3,"title":"intro"},{"first_sentence_index":4,"last_sentence_index":6,"title":"methods"},{"first_sentence_index":7,"last_sentence_index":8,"title":"conclusion"}]}
---
Now apply the same format.
INPUT: 51 sentences
Rule: group ADJACENT sentences into chunks (min 3, max 20 per chunk). Use 0-based indices. Ensure first_sentence_index ≤ last_sentence_index.
Return ONLY the JSON object.
JSON:
{assistant

{"chunks":[{"first_sentence_index":0,"last_sentence_index":5,"title":"introduction"},{"first_sentence_index":6,"last_sentence_index":12,"title":"background"},{"first_sentence_index":13,"last_sentence_index":19,"title":"methodology"},{"first_sentence_index":20,"last_sentence_index":28,"title":"results"},{"first_sentence_index":29,"last_sentence_index":34,"title":"discussion"},{"first_sentence_index":35,"last_sentence_index":40,"tit

In [ ]:
#@title (Optional) Preview a few records
import json
from pathlib import Path
from itertools import islice

path = Path("embeddings.jsonl")
if not path.exists():
    print("embeddings.jsonl 이 없습니다. 이전 셀을 먼저 실행하세요.")
else:
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(islice(f, 5)):
            row = json.loads(line)
            preview = row["text"][:120].replace("\n", " ")
            print(f"[{i}] {row['meta']['filetype']} #{row['meta']['chunk_index']} | {preview}")


[0] txt #0 | 인공지능은 이제 더 이상 연구실 안의 기술이 아닙니다. 우리는 매일같이 인공지능이 적용된 제품과 서비스를 사용하고 있습니다. 스마트폰의 얼굴 인식 기능, 온라인 쇼핑의 맞춤형 추천, 내비게이션의 교통 예측까지 모두 
[1] txt #1 | 하지만 인공지능의 발전이 긍정적인 효과만을 가져오는 것은 아닙니다. 자동화로 인해 많은 직업이 변화하거나 사라질 위험에 처해 있습니다. 예를 들어, 단순 반복 업무를 수행하던 직군은 인공지능 시스템에 의해 대체될 가
[2] txt #2 | 윤리적 문제도 중요한 화두입니다. 인공지능이 내린 결정이 언제나 공정하다고 볼 수는 없습니다. 데이터에 숨어 있는 편향이 알고리즘을 통해 증폭될 수 있기 때문입니다. 실제로 채용 과정에서 특정 성별이나 나이에 불리한
[3] txt #3 | 이러한 윤리적 논의 없이는 인공지능이 사회적 신뢰를 얻기 어렵습니다.  또한 인공지능은 국제 정치와 경제에도 큰 변화를 일으키고 있습니다. 미국과 중국은 인공지능을 차세대 전략 기술로 보고 막대한 투자를 하고 있습니
[4] txt #4 | 기후 변화, 전염병 대응, 우주 탐사와 같은 글로벌 문제를 해결하는 데 인공지능은 중요한 역할을 할 수 있습니다. 따라서 경쟁과 협력이 균형을 이루는 국제 환경 조성이 필요합니다.  마지막으로 우리는 인공지능을 어떻
